In [1]:
import pandas as pd
import numpy as np

In [2]:
import meteostat
from meteostat import Hourly, Stations, Point
from datetime import datetime, timedelta

In [3]:
df = pd.read_csv("../raw_data/fire_incidents_data.csv")

In [4]:
td = timedelta(hours = 1)

In [5]:
location = pd.merge(df["TFS_Alarm_Time"],df["Latitude"], right_index = True,
               left_index = True)
display(location.head())
print(len(location))

,TFS_Alarm_Time,Latitude
0,2018-02-24T21:04:29,43.686558
1,2018-02-24T21:24:43,43.766135
2,2018-02-25T13:29:59,43.743230
3,2018-02-25T14:13:39,43.708659
4,2018-02-25T18:20:43,43.718118


25860


In [6]:
dfDim = pd.concat([location, df["Longitude"]], axis = 1)
display(dfDim.head())

,TFS_Alarm_Time,Latitude,Longitude
0,2018-02-24T21:04:29,43.686558,-79.599419
1,2018-02-24T21:24:43,43.766135,-79.390039
2,2018-02-25T13:29:59,43.743230,-79.245061
3,2018-02-25T14:13:39,43.708659,-79.478062
4,2018-02-25T18:20:43,43.718118,-79.443184


In [7]:
dfDim = dfDim.dropna()
display(dfDim.head())
print(len(dfDim))

,TFS_Alarm_Time,Latitude,Longitude
0,2018-02-24T21:04:29,43.686558,-79.599419
1,2018-02-24T21:24:43,43.766135,-79.390039
2,2018-02-25T13:29:59,43.743230,-79.245061
3,2018-02-25T14:13:39,43.708659,-79.478062
4,2018-02-25T18:20:43,43.718118,-79.443184


25858


In [8]:
timeKey = dfDim["TFS_Alarm_Time"]
timeKey = timeKey.apply(lambda x: str(x))
timeKey = timeKey.apply(lambda x:x.replace("-", ""))
timeKey = timeKey.apply(lambda x:x.replace(":", ""))
timeKey = timeKey.apply(lambda x:x.replace(" ", ""))
timeKey = timeKey.apply(lambda x:x.replace("T", ""))
timeKey = timeKey.apply(lambda x:x[2:12])
display(timeKey.head())

0    1802242104
1    1802242124
2    1802251329
3    1802251413
4    1802251820
Name: TFS_Alarm_Time, dtype: object

In [9]:
latKey = dfDim["Latitude"]
longKey = dfDim["Longitude"]
display(latKey.head())
print(len(latKey))

0    43.686558
1    43.766135
2    43.743230
3    43.708659
4    43.718118
Name: Latitude, dtype: float64

25858


In [10]:
latKey = latKey.apply(lambda x: x-43)
display(latKey.head())

0    0.686558
1    0.766135
2    0.743230
3    0.708659
4    0.718118
Name: Latitude, dtype: float64

In [11]:
latKey = latKey.apply(lambda x:x*10000)
display(latKey.head())

0    6865.58177
1    7661.34782
2    7432.30192
3    7086.59308
4    7181.18427
Name: Latitude, dtype: float64

In [12]:
latKey = latKey.apply(lambda x: int(x))
display(latKey.head())

0    6865
1    7661
2    7432
3    7086
4    7181
Name: Latitude, dtype: int64

In [13]:
display(longKey.head())

0   -79.599419
1   -79.390039
2   -79.245061
3   -79.478062
4   -79.443184
Name: Longitude, dtype: float64

In [14]:
longKey = longKey.apply(lambda x:x+79)
display(longKey.head())

0   -0.599419
1   -0.390039
2   -0.245061
3   -0.478062
4   -0.443184
Name: Longitude, dtype: float64

In [15]:
longKey = longKey.apply(lambda x:abs(x))
display(longKey.head())

0    0.599419
1    0.390039
2    0.245061
3    0.478062
4    0.443184
Name: Longitude, dtype: float64

In [16]:
longKey = longKey.apply(lambda x:x*10000)
longKey = longKey.apply(lambda x:int(x))
display(longKey.head())

0    5994
1    3900
2    2450
3    4780
4    4431
Name: Longitude, dtype: int64

In [17]:
longKey = longKey.apply(lambda x: str(x))
latKey = latKey.apply(lambda x: str(x))
print(type(latKey[0]))

<class 'str'>


In [18]:
weatherKey = pd.concat([timeKey, latKey, longKey], axis = 1)
display(weatherKey.head())

,TFS_Alarm_Time,Latitude,Longitude
0,1802242104,6865,5994
1,1802242124,7661,3900
2,1802251329,7432,2450
3,1802251413,7086,4780
4,1802251820,7181,4431


In [19]:
weatherKey["Key"] = weatherKey.apply(lambda x:x["TFS_Alarm_Time"]+x["Latitude"]+x["Longitude"], axis = 1)
display(weatherKey.head())

,TFS_Alarm_Time,Latitude,Longitude,Key
0,1802242104,6865,5994,180224210468655994
1,1802242124,7661,3900,180224212476613900
2,1802251329,7432,2450,180225132974322450
3,1802251413,7086,4780,180225141370864780
4,1802251820,7181,4431,180225182071814431


In [20]:
weatherKey = weatherKey[["Key"]]
display(weatherKey.head())

,Key
0,180224210468655994
1,180224212476613900
2,180225132974322450
3,180225141370864780
4,180225182071814431


In [21]:
weatherData = pd.concat([weatherKey, dfDim], axis = 1)
display(weatherData.head())

,Key,TFS_Alarm_Time,Latitude,Longitude
0,180224210468655994,2018-02-24T21:04:29,43.686558,-79.599419
1,180224212476613900,2018-02-24T21:24:43,43.766135,-79.390039
2,180225132974322450,2018-02-25T13:29:59,43.743230,-79.245061
3,180225141370864780,2018-02-25T14:13:39,43.708659,-79.478062
4,180225182071814431,2018-02-25T18:20:43,43.718118,-79.443184


In [22]:
print(type(weatherData["TFS_Alarm_Time"][0]))

<class 'str'>


In [23]:
weatherData["TFS_Alarm_Time"] = weatherData["TFS_Alarm_Time"].apply(lambda x:datetime.strptime(x, '%Y-%m-%dT%H:%M:%S'))
display(weatherData.head())
print(type(weatherData["TFS_Alarm_Time"][0]))

,Key,TFS_Alarm_Time,Latitude,Longitude
0,180224210468655994,2018-02-24 21:04:29,43.686558,-79.599419
1,180224212476613900,2018-02-24 21:24:43,43.766135,-79.390039
2,180225132974322450,2018-02-25 13:29:59,43.743230,-79.245061
3,180225141370864780,2018-02-25 14:13:39,43.708659,-79.478062
4,180225182071814431,2018-02-25 18:20:43,43.718118,-79.443184


<class 'pandas._libs.tslibs.timestamps.Timestamp'>


In [24]:
weatherData["StartTime"] = weatherData["TFS_Alarm_Time"].apply(lambda dt:dt.replace(second=0, microsecond=0, minute=0))
display(weatherData.head())

,Key,TFS_Alarm_Time,Latitude,Longitude,StartTime
0,180224210468655994,2018-02-24 21:04:29,43.686558,-79.599419,2018-02-24 21:00:00
1,180224212476613900,2018-02-24 21:24:43,43.766135,-79.390039,2018-02-24 21:00:00
2,180225132974322450,2018-02-25 13:29:59,43.743230,-79.245061,2018-02-25 13:00:00
3,180225141370864780,2018-02-25 14:13:39,43.708659,-79.478062,2018-02-25 14:00:00
4,180225182071814431,2018-02-25 18:20:43,43.718118,-79.443184,2018-02-25 18:00:00


In [25]:
weatherData["EndTime"] = weatherData["StartTime"].apply(lambda x:x+td)
display(weatherData.head())

,Key,TFS_Alarm_Time,Latitude,Longitude,StartTime,EndTime
0,180224210468655994,2018-02-24 21:04:29,43.686558,-79.599419,2018-02-24 21:00:00,2018-02-24 22:00:00
1,180224212476613900,2018-02-24 21:24:43,43.766135,-79.390039,2018-02-24 21:00:00,2018-02-24 22:00:00
2,180225132974322450,2018-02-25 13:29:59,43.743230,-79.245061,2018-02-25 13:00:00,2018-02-25 14:00:00
3,180225141370864780,2018-02-25 14:13:39,43.708659,-79.478062,2018-02-25 14:00:00,2018-02-25 15:00:00
4,180225182071814431,2018-02-25 18:20:43,43.718118,-79.443184,2018-02-25 18:00:00,2018-02-25 19:00:00


In [26]:
dfPoints = weatherData.apply(lambda x:meteostat.Point(x["Latitude"], x["Longitude"]), axis = 1)
display(dfPoints.head())

0    <meteostat.interface.point.Point object at 0x0...
1    <meteostat.interface.point.Point object at 0x0...
2    <meteostat.interface.point.Point object at 0x0...
3    <meteostat.interface.point.Point object at 0x0...
4    <meteostat.interface.point.Point object at 0x0...
dtype: object

In [27]:
dfHourlyData = pd.concat([dfPoints, weatherData["StartTime"], weatherData["EndTime"]], axis = 1)
dfHourlyData = dfHourlyData.rename(columns = {0:"Points"})
display(dfHourlyData.head())

,Points,StartTime,EndTime
0,<meteostat.interface.point.Point object at 0x0...,2018-02-24 21:00:00,2018-02-24 22:00:00
1,<meteostat.interface.point.Point object at 0x0...,2018-02-24 21:00:00,2018-02-24 22:00:00
2,<meteostat.interface.point.Point object at 0x0...,2018-02-25 13:00:00,2018-02-25 14:00:00
3,<meteostat.interface.point.Point object at 0x0...,2018-02-25 14:00:00,2018-02-25 15:00:00
4,<meteostat.interface.point.Point object at 0x0...,2018-02-25 18:00:00,2018-02-25 19:00:00


In [28]:
dfHourlyData = dfHourlyData.loc[0:50]
print(len(dfHourlyData))

51


In [29]:
weatherData = weatherData.loc[0:50]
display(weatherData.head())
print(len(weatherData))

,Key,TFS_Alarm_Time,Latitude,Longitude,StartTime,EndTime
0,180224210468655994,2018-02-24 21:04:29,43.686558,-79.599419,2018-02-24 21:00:00,2018-02-24 22:00:00
1,180224212476613900,2018-02-24 21:24:43,43.766135,-79.390039,2018-02-24 21:00:00,2018-02-24 22:00:00
2,180225132974322450,2018-02-25 13:29:59,43.743230,-79.245061,2018-02-25 13:00:00,2018-02-25 14:00:00
3,180225141370864780,2018-02-25 14:13:39,43.708659,-79.478062,2018-02-25 14:00:00,2018-02-25 15:00:00
4,180225182071814431,2018-02-25 18:20:43,43.718118,-79.443184,2018-02-25 18:00:00,2018-02-25 19:00:00


51


In [30]:
weatherData["Weather"]= dfHourlyData.apply(lambda x:meteostat.Hourly(x['Points'], x['StartTime'],x['StartTime']), axis = 1)
display(weatherData.head())
# DO NOT RUNNN AGAIN

,Key,TFS_Alarm_Time,Latitude,Longitude,StartTime,EndTime,Weather
0,180224210468655994,2018-02-24 21:04:29,43.686558,-79.599419,2018-02-24 21:00:00,2018-02-24 22:00:00,<meteostat.interface.hourly.Hourly object at 0...
1,180224212476613900,2018-02-24 21:24:43,43.766135,-79.390039,2018-02-24 21:00:00,2018-02-24 22:00:00,<meteostat.interface.hourly.Hourly object at 0...
2,180225132974322450,2018-02-25 13:29:59,43.743230,-79.245061,2018-02-25 13:00:00,2018-02-25 14:00:00,<meteostat.interface.hourly.Hourly object at 0...
3,180225141370864780,2018-02-25 14:13:39,43.708659,-79.478062,2018-02-25 14:00:00,2018-02-25 15:00:00,<meteostat.interface.hourly.Hourly object at 0...
4,180225182071814431,2018-02-25 18:20:43,43.718118,-79.443184,2018-02-25 18:00:00,2018-02-25 19:00:00,<meteostat.interface.hourly.Hourly object at 0...


In [31]:
dummyData = {'time': ["2000-04-12 19:00:00"], 'temp':[-273], 'dwpt':[-273], 'rhum':[-273], 'prcp':[-273], 'snow':[-273], 'wdir':[-273], 'wspd':[-273], 'wpgt':[-273], 'pres':[-273], 'tsun':[-273], 'coco':[-273]}
dfDummy = pd.DataFrame(data = dummyData)

In [32]:
maxIndex = len(weatherData["Weather"])
print(maxIndex)
hourlyData = []
z = 0
for x in range(maxIndex):
    if x!=18863 and x!=19007:
        hourlyData.append(weatherData["Weather"][x].fetch())
        z = z+1
    else:
        hourlyData.append(weatherData["Weather"][0].fetch())
print(z)


51
51


In [33]:
z = 0
for x in range(len(hourlyData)):
    hourlyData[x] =hourlyData[x].reset_index()
    if hourlyData[x].empty:
        hourlyData[x] = dfDummy
        z = z+1
print(z)

0


In [34]:
dfHourly = pd.concat(hourlyData)
dfHourly = dfHourly.reset_index()
display(dfHourly.head())
print(len(dfHourly))

,index,time,temp,dwpt,rhum,prcp,snow,wdir,wspd,wpgt,pres,tsun,coco
0,0,2018-02-24 21:00:00,4.0,-3.5,58.0,NaN,NaN,70.0,9.4,NaN,1022.2,NaN,3.0
1,0,2018-02-24 21:00:00,4.0,-3.1,60.0,NaN,NaN,70.0,13.0,NaN,1022.4,NaN,NaN
2,0,2018-02-25 13:00:00,3.1,2.2,94.0,0.6,NaN,70.0,31.7,NaN,1001.7,NaN,7.0
3,0,2018-02-25 14:00:00,4.4,3.7,95.0,0.7,NaN,60.0,22.3,NaN,1001.5,NaN,NaN
4,0,2018-02-25 18:00:00,12.5,1.2,46.0,NaN,NaN,220.0,0.0,NaN,1005.0,NaN,NaN


51


In [35]:
dfHourly = dfHourly.drop(columns = {"time","tsun", "coco", "pres"})

In [36]:
display(dfHourly)
print(len(dfHourly))

,index,temp,dwpt,rhum,prcp,snow,wdir,wspd,wpgt
0,0,4.0,-3.5,58.0,NaN,NaN,70.0,9.4,NaN
1,0,4.0,-3.1,60.0,NaN,NaN,70.0,13.0,NaN
2,0,3.1,2.2,94.0,0.6,NaN,70.0,31.7,NaN
3,0,4.4,3.7,95.0,0.7,NaN,60.0,22.3,NaN
4,0,12.5,1.2,46.0,NaN,NaN,220.0,0.0,NaN
5,0,12.5,1.2,46.0,NaN,NaN,260.0,0.0,NaN
6,0,6.9,-5.5,41.0,NaN,NaN,250.0,0.0,NaN
7,0,1.3,-4.2,67.0,NaN,NaN,250.0,0.0,NaN
8,0,6.9,6.0,94.0,NaN,NaN,200.0,0.0,NaN
9,0,4.5,-4.5,52.0,NaN,NaN,260.0,22.3,NaN


51


In [37]:
weatherData = pd.concat([weatherData, dfHourly], axis = 1)
display(weatherData.head())

,Key,TFS_Alarm_Time,Latitude,Longitude,StartTime,EndTime,Weather,index,temp,dwpt,rhum,prcp,snow,wdir,wspd,wpgt
0,180224210468655994,2018-02-24 21:04:29,43.686558,-79.599419,2018-02-24 21:00:00,2018-02-24 22:00:00,<meteostat.interface.hourly.Hourly object at 0...,0,4.0,-3.5,58.0,NaN,NaN,70.0,9.4,NaN
1,180224212476613900,2018-02-24 21:24:43,43.766135,-79.390039,2018-02-24 21:00:00,2018-02-24 22:00:00,<meteostat.interface.hourly.Hourly object at 0...,0,4.0,-3.1,60.0,NaN,NaN,70.0,13.0,NaN
2,180225132974322450,2018-02-25 13:29:59,43.743230,-79.245061,2018-02-25 13:00:00,2018-02-25 14:00:00,<meteostat.interface.hourly.Hourly object at 0...,0,3.1,2.2,94.0,0.6,NaN,70.0,31.7,NaN
3,180225141370864780,2018-02-25 14:13:39,43.708659,-79.478062,2018-02-25 14:00:00,2018-02-25 15:00:00,<meteostat.interface.hourly.Hourly object at 0...,0,4.4,3.7,95.0,0.7,NaN,60.0,22.3,NaN
4,180225182071814431,2018-02-25 18:20:43,43.718118,-79.443184,2018-02-25 18:00:00,2018-02-25 19:00:00,<meteostat.interface.hourly.Hourly object at 0...,0,12.5,1.2,46.0,NaN,NaN,220.0,0.0,NaN


In [38]:
weatherData = weatherData.fillna(0)
display(weatherData.head())

,Key,TFS_Alarm_Time,Latitude,Longitude,StartTime,EndTime,Weather,index,temp,dwpt,rhum,prcp,snow,wdir,wspd,wpgt
0,180224210468655994,2018-02-24 21:04:29,43.686558,-79.599419,2018-02-24 21:00:00,2018-02-24 22:00:00,<meteostat.interface.hourly.Hourly object at 0...,0,4.0,-3.5,58.0,0.0,0.0,70.0,9.4,0.0
1,180224212476613900,2018-02-24 21:24:43,43.766135,-79.390039,2018-02-24 21:00:00,2018-02-24 22:00:00,<meteostat.interface.hourly.Hourly object at 0...,0,4.0,-3.1,60.0,0.0,0.0,70.0,13.0,0.0
2,180225132974322450,2018-02-25 13:29:59,43.743230,-79.245061,2018-02-25 13:00:00,2018-02-25 14:00:00,<meteostat.interface.hourly.Hourly object at 0...,0,3.1,2.2,94.0,0.6,0.0,70.0,31.7,0.0
3,180225141370864780,2018-02-25 14:13:39,43.708659,-79.478062,2018-02-25 14:00:00,2018-02-25 15:00:00,<meteostat.interface.hourly.Hourly object at 0...,0,4.4,3.7,95.0,0.7,0.0,60.0,22.3,0.0
4,180225182071814431,2018-02-25 18:20:43,43.718118,-79.443184,2018-02-25 18:00:00,2018-02-25 19:00:00,<meteostat.interface.hourly.Hourly object at 0...,0,12.5,1.2,46.0,0.0,0.0,220.0,0.0,0.0


In [39]:
weatherData = weatherData.drop(columns = {"index", "wpgt"})
display(weatherData.head())

,Key,TFS_Alarm_Time,Latitude,Longitude,StartTime,EndTime,Weather,temp,dwpt,rhum,prcp,snow,wdir,wspd
0,180224210468655994,2018-02-24 21:04:29,43.686558,-79.599419,2018-02-24 21:00:00,2018-02-24 22:00:00,<meteostat.interface.hourly.Hourly object at 0...,4.0,-3.5,58.0,0.0,0.0,70.0,9.4
1,180224212476613900,2018-02-24 21:24:43,43.766135,-79.390039,2018-02-24 21:00:00,2018-02-24 22:00:00,<meteostat.interface.hourly.Hourly object at 0...,4.0,-3.1,60.0,0.0,0.0,70.0,13.0
2,180225132974322450,2018-02-25 13:29:59,43.743230,-79.245061,2018-02-25 13:00:00,2018-02-25 14:00:00,<meteostat.interface.hourly.Hourly object at 0...,3.1,2.2,94.0,0.6,0.0,70.0,31.7
3,180225141370864780,2018-02-25 14:13:39,43.708659,-79.478062,2018-02-25 14:00:00,2018-02-25 15:00:00,<meteostat.interface.hourly.Hourly object at 0...,4.4,3.7,95.0,0.7,0.0,60.0,22.3
4,180225182071814431,2018-02-25 18:20:43,43.718118,-79.443184,2018-02-25 18:00:00,2018-02-25 19:00:00,<meteostat.interface.hourly.Hourly object at 0...,12.5,1.2,46.0,0.0,0.0,220.0,0.0


In [40]:
weatherData = weatherData.drop([18863,19007], inplace = True)
print(len(weatherData))

KeyError: '[18863, 19007] not found in axis'

In [ ]:
#print(weatherData.iloc[18863])

In [41]:
weatherData.drop(weatherData[weatherData['temp'] < -200].index, inplace = True)
display(weatherData.head())

,Key,TFS_Alarm_Time,Latitude,Longitude,StartTime,EndTime,Weather,temp,dwpt,rhum,prcp,snow,wdir,wspd
0,180224210468655994,2018-02-24 21:04:29,43.686558,-79.599419,2018-02-24 21:00:00,2018-02-24 22:00:00,<meteostat.interface.hourly.Hourly object at 0...,4.0,-3.5,58.0,0.0,0.0,70.0,9.4
1,180224212476613900,2018-02-24 21:24:43,43.766135,-79.390039,2018-02-24 21:00:00,2018-02-24 22:00:00,<meteostat.interface.hourly.Hourly object at 0...,4.0,-3.1,60.0,0.0,0.0,70.0,13.0
2,180225132974322450,2018-02-25 13:29:59,43.743230,-79.245061,2018-02-25 13:00:00,2018-02-25 14:00:00,<meteostat.interface.hourly.Hourly object at 0...,3.1,2.2,94.0,0.6,0.0,70.0,31.7
3,180225141370864780,2018-02-25 14:13:39,43.708659,-79.478062,2018-02-25 14:00:00,2018-02-25 15:00:00,<meteostat.interface.hourly.Hourly object at 0...,4.4,3.7,95.0,0.7,0.0,60.0,22.3
4,180225182071814431,2018-02-25 18:20:43,43.718118,-79.443184,2018-02-25 18:00:00,2018-02-25 19:00:00,<meteostat.interface.hourly.Hourly object at 0...,12.5,1.2,46.0,0.0,0.0,220.0,0.0


In [42]:
#remove hourly stats

In [43]:
weatherData = weatherData.drop(columns = {"Weather"})

In [44]:
weatherData.to_csv("Weather.csv", index = False)

In [ ]:
# problems: requesting data dropped some data from api
# the time range ie) 10 mins before + 10 mins after or?
# need to add comments
# cannot identify which fires are which anymore because of the change in times
#drop coco, tsun, pres

In [ ]:
#edit key rm stuff
#drop max wind speed and index
#remove invalid data
#remove the -273
